# Complicated Elo Predictor

This is meant to copy the way that Nate Silver Built his, from his methodology here: https://www.natesilver.net/p/sbcb-methodology


In [263]:
import pandas as pd
import yaml
import duckdb as db
import warnings
import os
from geopy.geocoders import Nominatim
import geopy
import time
import numpy as np

warnings.filterwarnings("ignore", category=UserWarning)

with open('config.yaml', 'r') as file:
    config_file = yaml.safe_load(file)
data_dir = config_file.get("data_dir")
output_dir = config_file.get("output_dir")

In [2]:
submission_df = pd.read_csv(f'{data_dir}/Kaggle/SampleSubmissionStage2.csv')

def extract_game_info(id_str):
    # Extract year and team_ids
    parts = id_str.split('_')
    year = int(parts[0])
    teamID1 = int(parts[1])
    teamID2 = int(parts[2])
    return year, teamID1, teamID2

submission_df[['Season', 'TeamID1', 'TeamID2']] = submission_df['ID'].apply(extract_game_info).tolist()

In [ ]:
# Some men's teams leave D1, so we *should* filter them out, but it breaks the basic model so I ignore it. The mean is still 1500.
# mensids = db.sql('FROM "./SourceData/Kaggle/MTeams.csv" WHERE LastD1Season = 2025').to_df()
mensids = db.sql('FROM "./SourceData/Kaggle/MTeams.csv"').to_df()
womensids = db.sql('FROM "./SourceData/Kaggle/WTeams.csv" ').to_df()

In [327]:
womens_results = pd.read_csv(f'{data_dir}/Kaggle/WRegularSeasonCompactResults.csv')
mens_results = pd.read_csv(f'{data_dir}/Kaggle/MRegularSeasonCompactResults.csv')

# # Add teams played column
# for teamID in mens_results['WTeamID'].unique():
#     mens_results.loc[mens_results.query(f"WTeamID == {teamID} | LTeamID == {teamID}").index, "Season_Game_Count"] = mens_results.query(f"WTeamID == {teamID} | LTeamID == {teamID}").groupby('Season').cumcount() + 1
# for teamID in womens_results['WTeamID'].unique():
#     womens_results.loc[womens_results.query(f"WTeamID == {teamID} | LTeamID == {teamID}").index, "Season_Game_Count"] = womens_results.query(f"WTeamID == {teamID} | LTeamID == {teamID}").groupby('Season').cumcount() + 1

In [331]:
# Initialize counters
game_counts = {}

def update_counts(row):
    season = row['Season']
    teams = [row['WTeamID'], row['LTeamID']]
    counts = []

    for team in teams:
        key = (season, team)
        if key in game_counts:
            game_counts[key] += 1
        else:
            game_counts[key] = 1
        counts.append(game_counts[key])

    return pd.Series(counts, index=['WTeam_Game_Count', 'LTeam_Game_Count'])

# Apply to DataFrame
mens_results[['WTeam_Game_Count', 'LTeam_Game_Count']] = mens_results.apply(update_counts, axis=1) 
womens_results[['WTeam_Game_Count', 'LTeam_Game_Count']] = womens_results.apply(update_counts, axis=1) 

In [ ]:
def get_home_locations(gender):
    #    """" Where gender is M of F for file matching"""
    cities = db.sql(f'''
    SELECT * FROM "{data_dir}/Kaggle/{gender}GameCities.csv" 
    JOIN "{data_dir}/Kaggle/Cities.csv" USING (CityID) 
    JOIN "{data_dir}/Kaggle/{gender}RegularSeasonCompactResults.csv" USING (Season, DayNum, WTeamID, LTeamID)
    WHERE City NOT IN ('MX', 'PR', 'VI','BA') AND
    Season != 2020
    '''
    ).to_df()

    home_games = cities.query('WLoc == "H"')
    lookup_table = home_games[['Season','WTeamID', 'City', 'State']].drop_duplicates().reset_index(drop=True)

    city_state_counts = lookup_table.groupby(['WTeamID', 'City', 'State']).size().reset_index(name='counts')
    most_frequent_city_state = city_state_counts.sort_values(['WTeamID', 'counts'], ascending=[True, False]).drop_duplicates('WTeamID')
    most_frequent_city_state = most_frequent_city_state.drop(columns=['counts'])
    most_frequent_city_state = most_frequent_city_state.sort_values(by=['WTeamID'])
    most_frequent_city_state.rename(columns={'WTeamID': 'TeamID'}, inplace=True)
    return most_frequent_city_state

In [216]:
# womens
W_Home_Lookup = get_home_locations('W')
# These three don't have any home game entries:
all_missing_entries = pd.DataFrame([
    {'TeamID': 3128, 'City': 'Birmingham', 'State': 'AL'},
    {'TeamID': 3289, 'City': 'Atlanta', 'State': 'GA'},
    {'TeamID': 3445, 'City': 'Winston-Salem', 'State': 'NC'}
])
W_Home_Lookup = pd.concat([W_Home_Lookup, all_missing_entries], ignore_index=True)
W_Home_Lookup['gender'] = 'W'

In [217]:
# mens
M_Home_Lookup = get_home_locations('M')
all_missing_entries = pd.DataFrame([
    {'TeamID': 1109, 'City': 'San Diego', 'State': 'CA'},
    {'TeamID': 1118, 'City': 'Savannah', 'State': 'GA'},
    {'TeamID': 1121, 'City': 'Augusta', 'State': 'GA'},
    {'TeamID': 1128, 'City': 'Birmingham', 'State': 'AL'},
    {'TeamID': 1134, 'City': 'Brooklyn', 'State': 'NY'},
    {'TeamID': 1215, 'City': 'Abilene', 'State': 'TX'},
    {'TeamID': 1289, 'City': 'Atlanta', 'State': 'GA'},
    {'TeamID': 1302, 'City': 'Chicago', 'State': 'IL'},
    {'TeamID': 1327, 'City': 'Oklahoma City', 'State': 'OK'},
    {'TeamID': 1432, 'City': 'Utica', 'State': 'NY'},
    {'TeamID': 1446, 'City': 'Canyon', 'State': 'TX'}
])
M_Home_Lookup = pd.concat([M_Home_Lookup, all_missing_entries], ignore_index=True)
M_Home_Lookup['gender'] = 'M'

In [159]:
# Concatenate M_Home_Lookup and W_Home_Lookup dataframes
Home_Lookup = pd.concat([M_Home_Lookup, W_Home_Lookup], ignore_index=True)
geolocator = Nominatim(user_agent="marshall")

def get_lat_lon(city, state):
    time.sleep(1)
    print(f"Geocoding {city}, {state}")
    location = geolocator.geocode(f"{city}, {state}",timeout=50)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None


In [ ]:
# This takes 15 minutes so save it as a csv to avoid redoing it
if not os.path.exists(f'{data_dir}/derived/Home_Lookup.csv'):
    Home_Lookup[['Latitude', 'Longitude']] = Home_Lookup.apply(lambda row: pd.Series(get_lat_lon(row['City'], row['State'])), axis=1)
    Home_Lookup.to_csv(f'{data_dir}/derived/Home_Lookup.csv', index=False)
else:
    Home_Lookup = pd.read_csv(f'{data_dir}/derived/Home_Lookup.csv')

In [235]:
def travel_distance(row):
    W_team_location = Home_Lookup[(Home_Lookup['TeamID'] == row['WTeamID'])]
    L_team_location = Home_Lookup[(Home_Lookup['TeamID'] == row['LTeamID'])]

    away_distance = geopy.distance.distance(
        (W_team_location.Latitude.values[0], W_team_location.Longitude.values[0]),
        (L_team_location.Latitude.values[0], L_team_location.Longitude.values[0])).miles

    if row1.WLoc == 'N':
        win_distance = 0
        lose_distance = 0
    elif row1.WLoc == 'H':
        win_distance = 0
        lose_distance = away_distance
    elif row1.WLoc == 'A':
        win_distance = away_distance
        lose_distance = 0

    return win_distance, lose_distance

def distance_to_elo_impact(win_distance, lose_distance):
    # 8 * m^(⅓)
    win_impact = 8 * (win_distance ** (1/3))
    lose_impact = 8 * (lose_distance ** (1/3))
    return win_impact, lose_impact

# How to use it
row1 = mens_results.loc[8]

win_distance, lose_distance = travel_distance(row1)
distance_to_elo_impact(win_distance, lose_distance)

(0.0, 40.60880133703453)

In [266]:
# Point 1: Margin of victory: 
# Specifically, the margin of victory factor is calculated as (3 + s) ^ .85, where s is the scoring differential.
# I assume this should be added to the change in elo for the winner/loser
# I made up my own here

def point_differential_scaler(row, mov_average=12):
#   League average margin of victory is 12.
#   If you score less than that, it doesn't impact your elo
#   If you score more than that, it scales up the elo change by a multiplier
#   double the average (12) is 2x multiplier, and it decreases at nat log rate
    mov = (row['WScore']- row['LScore'])
    if mov < mov_average:
        k_multiplier = 1
    else:
        k_multiplier = 1 + (1 / np.log(mov_average+1)) * np.log(np.abs(mov-mov_average) + 1)
    return k_multiplier

In [270]:
# This will be a 10 point scaler to ELO
# This could be improved by looking at the values of the losing teams, because as it is now it only uses the winning teams location
# For calculating margins of victory, one point in a basketball game equals approximately 27 Elo points.
# There shouldn't be a negative home field impact, even if the numbers say so

def home_field_advantage_calculator(teams_df, season_cutoff = 2000, values_cutoff = 5):
    # Filter mens_results to only include seasons from the year 2000 onwards
    teams_df_filtered = teams_df[teams_df['Season'] >= season_cutoff]

    # Calculate the average points scored by each team at home and away (including neutral courts)
    home_points = teams_df_filtered[teams_df_filtered['WLoc'] == 'H'].groupby('WTeamID')['WScore'].mean().reset_index()
    home_points.columns = ['TeamID', 'HomePoints']

    away_points = teams_df_filtered[teams_df_filtered['WLoc'].isin(['A', 'N'])].groupby('WTeamID')['WScore'].mean().reset_index()
    away_points.columns = ['TeamID', 'AwayPoints']

    # Merge the home and away points dataframes
    points_comparison = pd.merge(home_points, away_points, on='TeamID', how='inner')

    # Calculate the expected additional points at home
    points_comparison['HomeAdvantage'] = points_comparison['HomePoints'] - points_comparison['AwayPoints']
    points_comparison['HomeAdvantage'] = points_comparison['HomeAdvantage'].clip(lower=0, upper=values_cutoff)

    return points_comparison[['TeamID', 'HomeAdvantage']]

HFA_df = home_field_advantage_calculator(mens_results)

In [272]:
# Calculate home field advantage for men's and women's results
mens_HFA = home_field_advantage_calculator(mens_results)
womens_HFA = home_field_advantage_calculator(womens_results)

# Add a column to distinguish between men's and women's data
mens_HFA['Gender'] = 'M'
womens_HFA['Gender'] = 'W'

# Concatenate the two dataframes
combined_HFA = pd.concat([mens_HFA, womens_HFA], ignore_index=True)
combined_HFA.to_csv(f'{data_dir}/derived/HomeFieldAdvantage.csv', index=False)

In [421]:
# Mean reversion:
# Empirically, the degree of mean reversion from year to year is growing — in other words, 
# teams are less likely to sustain their success — probably because the best players typically
# leave for the NBA after one or two years in college; even elite programs now rarely maintain
# dominance with the same core of talent. Currently, a team’s rating is reverted by 30-35 percent
# toward the mean at the start of each new season.
# Should revert to the mean of the conference ratings
# The baysian model uses pre-season ratings, but this would be a bit harder to implement
# "partly on preseason rankings in the AP (media) and Coaches Polls."

mens_conf = pd.read_csv(f'{data_dir}/Kaggle/MTeamConferences.csv')
womens_conf = pd.read_csv(f'{data_dir}/Kaggle/WTeamConferences.csv')
all_conf = pd.concat([mens_conf, womens_conf], ignore_index=True)

# results_season = mens_results[mens_results['Season'] == 2000]
# elo_ratings = {team_id: 1500 for team_id in mensids['TeamID'].unique()}
# df = pd.DataFrame(list(elo_ratings.items()), columns=['TeamID', 'Elo'])
# conference = db.sql(f"SELECT Elo, COALESCE(ConfAbbrev,'unknown') as conf FROM df LEFT JOIN all_conf USING (TeamID) WHERE Season == 2000").to_df();
# conference_mean = conference.groupby('conf')['Elo'].transform('mean')
# elo_ratings = {team_id: (1-mean_reversion) * elo + (mean_reversion) * conf_mean for team_id, elo, conf_mean in zip(elo_ratings.keys(), elo_ratings.values(), conference_mean)}

In [336]:
# K factor: Specifically, we use a k-factor of 38; this number has no intrinsic meaning and is derived empirically.
# However, the k-factor is up to 50 percent higher (so, up to a k-factor of 56) for early-season games, 
# with this diminishing linearly to a k-factor of 38 until a team plays roughly the 20th game of its season.

def k_factor_calculator(game_number, k_factor_start=56, k_factor_end = 38):
    """ takes the input of the game number and returns the k-factor for the game
    :param game_number: int, the number of the game
    :param k_factor_start: int, the starting k-factor for the first game
    :param k_factor_end: int, the ending k-factor for the last game
    :return: int, the k-factor for the game
    """
    if game_number < k_factor_start-k_factor_end:
        k_factor = k_factor_start - (game_number - 1)
    else:
        k_factor = 38
    return k_factor

In [ ]:
# NCAA tournament games also receive an additional multiple of 1.25x, tantamount to a k-factor of 47.5
# Not sure if I want to look at NCAA games as well, but maybe
# An additional multiplier of 1.07x is applied to the Elo ratings
# difference between the teams in forecasting margins of victory and win probabilities in the tournament. 

In [ ]:
# WOMEN:
# less mean-reversion from season to season
# Home court advantage tends to be slightly less in the women’s game
# the ratio of Elo rating point differences to the point spread is about 25:1 for women as opposed to 27:1 for men

In [67]:
# Composite with https://kenpom.com/? (1.5x)
# Composite with https://sonnymoorepowerratings.com/m-basket.htm?
# https://www.espn.com/mens-college-basketball/bpi? 
# Massey ratings: https://masseyratings.com/cb2024/ncaad1/ratings

# Womens:
# https://herhoopstats.com/stats/ncaa/research/team_single_seasons/?min_season=2025&max_season=2025&division=1&games=all&criteria0=hhs_net_rtg&comp0=ge&threshold0=-100&stats_to_show=summary_advanced&submit=true

SyntaxError: invalid syntax (1495644420.py, line 6)

In [292]:
def get_advantage(team_id, df, scalar=26):
#   pass in Home_Lookup.csv
#   Set the scalar to whatever you want to correspond to points per elo
#   the ratio of Elo rating point differences to the point spread is about 25:1 for women as opposed to 27:1 for men
#   So I use the mean of the two
    if team_id not in df['TeamID'].values:
        advantage = 2.12698 * scalar # Same as the mean, but trying to speed up performance
        # advantage = df['HomeAdvantage'].mean()
    else:
        advantage = df.loc[df['TeamID'] == team_id, 'HomeAdvantage'].values[0] * scalar
    return advantage

In [318]:
mens_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Season_Game_Count
0,1985,20,1228,81,1328,64,N,0,NaN
1,1985,25,1106,77,1354,70,H,0,NaN
2,1985,25,1112,63,1223,56,H,0,NaN
3,1985,25,1165,70,1432,54,H,0,NaN
4,1985,25,1192,86,1447,74,H,0,NaN
...,...,...,...,...,...,...,...,...,...
192492,2025,120,1433,71,1182,62,A,0,NaN
192493,2025,120,1436,79,1107,71,H,0,NaN
192494,2025,120,1438,60,1199,57,H,0,NaN
192495,2025,120,1452,71,1428,69,A,0,NaN


In [459]:
k_set = 30 # K-factor for Elo rating
# K-factor determines how much the Elo rating changes after each game
# Higher K-factor means more volatility, Lower K-factor means more stable Elo ratings
# The K-factor is usually set between 10 and 40
initial_elo_set = 1500 # Initial Elo rating for all teams
# Initial Elo rating is usually set to 1500, but can be set to any value
mean_reversion = .3 # Mean reversion ratio for Elo rating
# Mean reversion ratio determines how much the Elo rating reverts to the initial/mean Elo rating after each season
# Used to reflect the turnover in a sports team, higher mean reversion means more turnover in the team
# I chose to go with 25% of the elo returns to mean as a starting point
home_field_df = pd.read_csv(f'{data_dir}/derived/HomeFieldAdvantage.csv')

def update_elo(winner_elo, loser_elo, k=k_set):

    expected_win = 1 / (1 + 10**((loser_elo - winner_elo) / 400))
    elo_change = + k * (1 - expected_win)
    return elo_change

def run_basic_elo(season_results_df, ids_df):

    seasons_array = sorted(season_results_df['Season'].unique())
    initial_elo = initial_elo_set 
    elo_ratings = {team_id: initial_elo for team_id in ids_df['TeamID'].unique()}
    df = pd.DataFrame(list(elo_ratings.items()), columns=['TeamID', 'Elo'])

    for i in seasons_array:
        results_season = season_results_df[season_results_df['Season'] == i]
        active_teams = set(results_season['WTeamID']).union(results_season['LTeamID'])

        # Preserve last season's ratings before overwriting
        prev_elo_ratings = elo_ratings.copy()

        elo_ratings = {
            team_id: prev_elo_ratings.get(team_id, initial_elo) for team_id in active_teams
        }
        print(f"Running season {i}")
        
        for index, row in results_season.iterrows():
            winner = row['WTeamID']
            loser = row['LTeamID']

            # Add Travel Adjustment
            # Doesn't yet work for neutral games, uses 0's
            win_distance, lose_distance = travel_distance(row)
            win_distance_impact, lose_distance_impact = distance_to_elo_impact(win_distance, lose_distance)
            winner_elo = elo_ratings[winner] + win_distance_impact
            loser_elo = elo_ratings[loser] + lose_distance_impact
            
            # Add Home Field Advantage
            if row['WLoc'] == 'H':
                home_advantage = get_advantage(winner, home_field_df, scalar=26)
                winner_elo = elo_ratings[winner] + home_advantage
            elif row['WLoc'] == 'A':
                home_advantage = get_advantage(loser, home_field_df, scalar=26)
                loser_elo = elo_ratings[loser] + home_advantage
            elif row['WLoc'] == 'N':
                winner_elo = elo_ratings[winner] + 0
                loser_elo = elo_ratings[loser] + 0

            # Add Point Differential
            k_coeff = point_differential_scaler(row, mov_average=12)
            
            # Calculate the K-factor for the game, this is a linear function of the game number
            game_number_mean = (row['WTeam_Game_Count']+ row['LTeam_Game_Count'])/2
            k = k_factor_calculator(game_number_mean, 56, 38)
            
            elo_change = update_elo(winner_elo, loser_elo, k * k_coeff)
            # the winner and loser elo variables are *adjusted* elo (by home field, travel, etc),
            # so we need to apply the change to the dictionary values instead
            winner_elo_old = elo_ratings[winner]
            loser_elo_old = elo_ratings[loser]
            # Add the elo change
            new_winner_elo = winner_elo_old + elo_change
            new_loser_elo = loser_elo_old - elo_change

            elo_ratings[winner] = new_winner_elo
            elo_ratings[loser] = new_loser_elo

        df = pd.DataFrame(list(elo_ratings.items()), columns=['TeamID', 'Elo'])

        # Revert to conference mean
        conference = db.sql(f"""
            SELECT TeamID, Elo, COALESCE(ConfAbbrev,'unknown') as conf 
            FROM df LEFT JOIN all_conf USING (TeamID) 
            WHERE Season == {i}
        """).to_df()

        # This divergence bit is an attempt to counteract teams that drop out and then skew the mean
        conf_means = conference.groupby('conf')['Elo'].mean().to_dict()
        league_mean = df['Elo'].mean()
        divergence = initial_elo_set-league_mean

        elo_ratings = {
            row.TeamID: ((1 - mean_reversion) * row.Elo + mean_reversion * conf_means.get(row.conf, initial_elo)) * (1+divergence/initial_elo_set)
            for row in conference.itertuples()
        }
        print(f"Mean Elo for season {i}: {league_mean}")
        print(f"divergence from mean: {divergence}")
        # This commented out line does a league wide averege (1500) instead of a conference average
        # elo_ratings = {team_id: (1-mean_reversion) * elo + (mean_reversion) * initial_elo for team_id, elo in elo_ratings.items()}
        df = pd.DataFrame(list(elo_ratings.items()), columns=['TeamID', 'Elo'])
        
    return df

mens_elo = run_basic_elo(mens_results, mensids)
womens_elo = run_basic_elo(womens_results, womensids)

Running season 1985
Mean Elo for season 1985: 1499.9999999999998
divergence from mean: 2.2737367544323206e-13
Running season 1986
Mean Elo for season 1986: 1500.5091853231302
divergence from mean: -0.5091853231301684
Running season 1987
Mean Elo for season 1987: 1500.2039268464043
divergence from mean: -0.2039268464043289
Running season 1988
Mean Elo for season 1988: 1501.8625229116376
divergence from mean: -1.8625229116375976
Running season 1989
Mean Elo for season 1989: 1499.9977110180591
divergence from mean: 0.002288981940864687
Running season 1990
Mean Elo for season 1990: 1502.6707373498723
divergence from mean: -2.6707373498722973
Running season 1991
Mean Elo for season 1991: 1500.850710652779
divergence from mean: -0.8507106527790711
Running season 1992
Mean Elo for season 1992: 1502.2645098333226
divergence from mean: -2.2645098333225633
Running season 1993
Mean Elo for season 1993: 1501.6379420362532
divergence from mean: -1.6379420362532073
Running season 1994
Mean Elo for s

In [460]:
All_elo = pd.concat([mens_elo, womens_elo], ignore_index=True)

warnings.filterwarnings('ignore')
# Create a dictionary for quick lookup of ELO ratings by TeamID
elo_dict = All_elo.set_index('TeamID')['Elo'].to_dict()

# Map the ELO ratings to the TeamID1 column in the submission_df
submission_df['TeamID1_Elo'] = submission_df['TeamID1'].map(elo_dict)
submission_df['TeamID2_Elo'] = submission_df['TeamID2'].map(elo_dict)

# Fill missing values with 9999 - these would be teams that aren't in the nate database of mismatches in names
submission_df['TeamID1_Elo'].fillna(9999, inplace=True)
submission_df['TeamID2_Elo'].fillna(9999, inplace=True)

# Check the result, this should be 0
assert len(submission_df.query('TeamID1_Elo == 9999 or TeamID2_Elo == 9999')) == 0, "There are teams with missing ELO ratings"

In [461]:
# Basic ELO win probability calculation
def calc_elo_win(A, B):
    awin = 1 / (1 + 10**( (B - A) / 400))
    return(awin)

# differences in team strength are typically more apparent in the tournament, 
# and the model accounts for this, too. An additional multiplier of 1.07x is applied 
# to the Elo ratings difference between the teams in forecasting margins of 
# victory and win probabilities in the tournament.
def calc_elo_win_tourney(A, B, boost=1.07):
    awin = 1 / (1 + 10**( (B - A) * boost / 400))
    return(awin)

submission_df['Team1_win_prob'] = submission_df.apply(lambda x: calc_elo_win_tourney(x['TeamID1_Elo'], x['TeamID2_Elo']), axis=1)

In [463]:
Output = submission_df[['ID', 'Team1_win_prob']].rename(columns={'Team1_win_prob': 'Pred'})
Output.to_csv(f'{output_dir}/ComplexEloProbs.csv', index=False)

In [ ]:
Output_me = Output.copy()
Output_ns = pd.read_csv(f'{output_dir}/NateEloProbs.csv')

In [363]:
Output_merge = Output_me.merge(Output_ns, on='ID', how='left')
Output_merge['diff'] = Output_merge['Pred_x'] - Output_merge['Pred_y']
